[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobinL/uk_address_matcher/blob/main/match_example_data.ipynb)


In [9]:
!pip install --pre uk_address_matcher
!pip install git+https://github.com/moj-analytical-services/splink.git@2580-improve-runtimes-but-pushing-up-common-case-statements-into-precomputed-values


In [1]:
import duckdb
import pandas as pd

from uk_address_matcher.post_linkage.analyse_results import (
    distinguishability_summary,
)
from uk_address_matcher.post_linkage.identify_distinguishing_tokens import (
    improve_predictions_using_distinguishing_tokens,
)
from uk_address_matcher import clean_data_using_precomputed_rel_tok_freq, get_linker
import time

pd.options.display.max_colwidth = 1000

pd.options.display.max_colwidth = 1000

# -----------------------------------------------------------------------------
# Step 1: Load in some example data.  If using your own data, it must be in
# the same format as the example data.
# -----------------------------------------------------------------------------
# Any additional columns should be retained as-is by the cleaning code

p_ch = "./example_data/companies_house_addresess_postcode_overlap.parquet"
p_fhrs = "./example_data/fhrs_addresses_sample.parquet"

con = duckdb.connect(database=":memory:")

df_ch = con.read_parquet(p_ch).order("postcode")
df_fhrs = con.read_parquet(p_fhrs).order("postcode")

# Display length of the dataset
print(f"Length of FHRS dataset: {len(df_fhrs.df()):,.0f}")
print(f"Length of Companies House dataset: {len(df_ch.df()):,.0f}")

display(df_fhrs.limit(5).df())
display(df_ch.limit(5).df())


Length of FHRS dataset: 5,000
Length of Companies House dataset: 21,952


,unique_id,source_dataset,address_concat,postcode
0,300681,fhrs,AMERICA ROAD EARLS COLNE COLCHESTER,C06 2LB
1,300880,fhrs,THE AIRFIELD EARLS COLNE COLCHESTER ESSEX,C06 2NS
2,300359,fhrs,THE AIRFIELD EARLS COLNE COLCHESTER ESSEX,C06 2NS
3,300752,fhrs,9A BRIDGE STREET BURES HAMLET SUDBURY SUFFOLK,C08 5AD
4,300821,fhrs,MOUNT HILL HALSTEAD ESSEX,C09 1AA


,unique_id,source_dataset,address_concat,postcode
0,00702725,companies_house,UNIT 11 SPITFIRE DRIVE EARLS COLNE BUSINESS PARK EARLS COLNE ESSEX,C06 2NS
1,00988738,companies_house,BARKER HOUSE PHOENIX ROAD HAVERHILL,CB9 7AE
2,13021290,companies_house,CHESTNUT LODGE HELIONS BUMPSTEAD HAVERHILL SUFFOLK,CB9 7AF
3,06932985,companies_house,"ESSEX COTTAGE PALE GREEN, HELIONS BUMPSTEAD HAVERHILL SUFFOLK",CB9 7AF
4,15507930,companies_house,HILLTOP VILLA PALE GREEN HELIONS BUMPSTEAD HAVERHILL,CB9 7AF


In [2]:
# -----------------------------------------------------------------------------
# Step 2: Clean the data/feature engineering to prepare for matching model
# -----------------------------------------------------------------------------

df_fhrs_clean = clean_data_using_precomputed_rel_tok_freq(df_fhrs, con=con)
df_ch_clean = clean_data_using_precomputed_rel_tok_freq(df_ch, con=con)


In [4]:
linker = get_linker(
    df_addresses_to_match=df_fhrs_clean,
    df_addresses_to_search_within=df_ch_clean,
    con=con,
    include_full_postcode_block=True,
    additional_columns_to_retain=["original_address_concat"],
)

df_predict = linker.inference.predict(
    threshold_match_weight=-50, experimental_optimisation=True
)
df_predict_ddb = df_predict.as_duckdbpyrelation()

Blocking time: 0.01 seconds
Predict time: 1.15 seconds


In [ ]:
start_time = time.time()
df_predict_improved = improve_predictions_using_distinguishing_tokens(
    df_predict=df_predict_ddb,
    con=con,
    match_weight_threshold=-20,
)

df_predict_improved.show(max_width=500, max_rows=5)

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

┌─────────────┬─────────────┬────────────────┬───────────────────────┬─────────────────────┬────────────────────────────┬───────────────────────┬───────────────────────────────────┬───────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────────────────────────────────────────────┬────────────┐
│ unique_id_r │ unique_id_l │ dist_tok_match │ match_weight_original │    match_weight     │ match_probability_original │   match_probability   │ canonical_distinguishing_tokens_1 │ messy_distinguishing_tokens_1 │                              original_address_concat_l                              │ postcode_l │             original_address_concat_r              │ postcode_r │
│   varchar   │   varchar   │    boolean     │        double         │       double        │           double           │        double         │             varchar[]             │           varchar[]           │                             

In [6]:
dsum_1 = distinguishability_summary(
    df_predict=df_predict_ddb, df_addresses_to_match=df_fhrs_clean, con=con
)
dsum_1.show(max_width=500, max_rows=20)

dsum_2 = distinguishability_summary(
    df_predict=df_predict_improved, df_addresses_to_match=df_fhrs_clean, con=con
)
dsum_2.show(max_width=500, max_rows=20)


┌─────────────────────────────┬───────┬────────────┐
│ distinguishability_category │ count │ percentage │
│           varchar           │ int64 │  varchar   │
├─────────────────────────────┼───────┼────────────┤
│ 01: One match only          │   757 │ 15.14%     │
│ 02: Distinguishability > 10 │   601 │ 12.02%     │
│ 03: Distinguishability > 5  │   116 │ 2.32%      │
│ 04: Distinguishability > 1  │   449 │ 8.98%      │
│ 05: Distinguishability > 0  │   126 │ 2.52%      │
│ 06.: Distinguishability = 0 │  1781 │ 35.62%     │
│ 99: No match                │  1170 │ 23.40%     │
└─────────────────────────────┴───────┴────────────┘

┌─────────────────────────────┬───────┬────────────┐
│ distinguishability_category │ count │ percentage │
│           varchar           │ int64 │  varchar   │
├─────────────────────────────┼───────┼────────────┤
│ 01: One match only          │   758 │ 15.16%     │
│ 02: Distinguishability > 10 │   830 │ 16.60%     │
│ 03: Distinguishability > 5  │   388 │ 7.76%

In [8]:
from uk_address_matcher.post_linkage.analyse_results import distinguishability_by_id

distinguishability_by_id(df_predict=df_predict_improved, df_addresses_to_match=df_fhrs_clean, con=con).df().sample(10)


,unique_id_r,unique_id_l,original_address_concat_l,postcode_l,match_probability,match_weight,distinguishability,distinguishability_category,original_address_concat_r,postcode_r
4786,300821,None,None,None,NaN,NaN,NaN,99: No match,MOUNT HILL HALSTEAD ESSEX,C09 1AA
2939,55538,13384142,12 RUNWELL RAOD WICKFORD ESSEX,SS11 7AB,0.000351,-16.473931,0.000000,06.: Distinguishability = 0,22-28 RUNWELL ROAD WICKFORD ESSEX,SS11 7AB
4496,108578,None,None,None,NaN,NaN,NaN,99: No match,MILTON ERNEST LOWER SCHOOL THURLEIGH ROAD MILTON ERNEST BEDFORD,MK44 1RF
845,1689688,15110190,FLAT 265 ROMAN ROAD MOUNTNESSING BRENTWOOD,CM15 0UH,0.984960,11.033219,22.507150,02: Distinguishability > 10,265 ROMAN ROAD MOUNTNESSING ESSEX,CM15 0UH
2832,1612840,08277743,THE WHITE HART INN 15 HIGH STREET HALSTEAD ESSEX,CO9 2AA,0.000351,-16.473931,0.000000,06.: Distinguishability = 0,7 HIGH STREET HALSTEAD ESSEX,CO9 2AA
4511,1552285,None,None,None,NaN,NaN,NaN,99: No match,UNIT 4 WESTGATE PARK FODDERWICK BASILDON ESSEX,SS14 1WP
931,1558364,15460542,OFFICE F 12 THE PLAZA 1 RUTHERFORD PARK GREAT NOTLEY BRAINTREE ESSEX,CM77 7AU,0.954001,9.374331,13.758781,02: Distinguishability > 10,S 12 THE PLAZA ENTERPRISE AND INNOVATION CENTRE 1 RUTHERFORD PARK GREAT NOTLEY BRAINTREE,CM77 7AU
3630,1646994,14354135,WALNUT TREE HOUSE HALL STREET LONG MELFORD SUDBURY,CO10 9JG,0.000088,-18.473931,0.000000,06.: Distinguishability = 0,THE BULL HOTEL THE BULL HOTEL HALL STREET LONG MELFORD SUDBURY,CO10 9JG
1214,1576531,14282879,UNIT 4 CHURCH ARCADE BEDFORD,MK40 1LQ,0.733768,6.462643,22.936574,02: Distinguishability > 10,4 CHURCH ARCADE BEDFORD,MK40 1LQ
3955,1691544,None,None,None,NaN,NaN,NaN,99: No match,VENTENFALL PROJECT - MURPHY CONSTRUCTION SITE SCARNING NORFOLK,NR19 2QN


In [13]:


sql = """
SELECT * FROM df_predict_ddb WHERE match_weight > 0
QUALIFY row_number() OVER (PARTITION BY unique_id_l ORDER BY match_weight DESC) = 1
order by random()
limit 3
"""

recs = con.sql(sql).df().to_dict(orient="records")


for rec in recs:
    print("-" * 80)
    print(rec["unique_id_l"], rec["original_address_concat_l"])
    print(rec["unique_id_r"], rec["original_address_concat_r"])
    display(linker.visualisations.waterfall_chart([rec]))


--------------------------------------------------------------------------------
12261043 OLIVERS BARN MALDON ROAD WITHAM ESSEX
300994 OLIVERS FARM NURSERIES MALDON ROAD WITHAM ESSEX


alt.LayerChart(...)

--------------------------------------------------------------------------------
14928577 WOODLAND MANOR GREEN LANE CLAPHAM BEDFORD BEDFORDSHIRE
1082241 WOODLAND MANOR GREEN LANE CLAPHAM BEDFORD


alt.LayerChart(...)

--------------------------------------------------------------------------------
12539680 BEDFORD HEIGHTS BRICKHILL DRIVE BEDFORD
846804 BEDFORD HEIGHTS BRICKHILL DRIVE BEDFORD


alt.LayerChart(...)